In [1]:
import os
import gc   # garbage collector
import time
import pandas as pd
import numpy as np

import rasterio
import rioxarray as rioxr

import dask_gateway
import dask.array as da

from joblib import load

import geopandas as gpd
from shapely.geometry import mapping

from scipy.ndimage import median_filter

# custom modules
import raster_to_features as rf
import A_data_sampling_workflow.sample_rasters as sr

from skimage.feature import graycomatrix, graycoprops

from shapely.geometry import Polygon

In [2]:
# year for which to predict iceplant locations
year = 2020

# whether to apply median filter to raster
filter_rasters = True

# whether to save rasters
save_rasters = True
prefix = 'glcmTRIAL'

# whether to print processing info at runtime
verbose = True

save_processing_times = True

entropy_r = 5
box_side = entropy_r*2 +1

In [3]:


# **************************************************************
# load pre-trained random forest classifier
rfc = load(prefix+'_rfc.joblib') 


# **************************************************************
# prepare folder to save rasters
if save_rasters:
    fp = os.path.join(os.getcwd(), 'processing_results')
    if os.path.exists(fp) == False:
        os.mkdir(fp)
    if filter_rasters:
        fp = os.path.join(fp, prefix+'_filter_clip_preds_' + str(year))
    else:
        fp = os.path.join(fp, prefix+'_clip_preds_' + str(year))
    if os.path.exists(fp) == False:
        os.mkdir(fp)

In [5]:
# # Campus Point only
# ca_m_3411934_sw_11_060_20200521
# reduce_box = Polygon([
#     [-119.8454926305,34.4047504708],
#     [-119.8436963214,34.4047504708],
#     [-119.8436963214,34.4062633726],
#     [-119.8454926305,34.4062633726],
#     [-119.8454926305,34.4047504708]])

# slough road
# reduce_box = Polygon([
#     [-119.8767816114,34.4116240455],
#     [-119.8749853022,34.4116240455],
#     [-119.8749853022,34.4131368229],
#     [-119.8767816114,34.4131368229],
#     [-119.8767816114,34.4116240455]])

# chase palm park
#ca_m_3411935_se_11_060_20200521
reduce_box = Polygon([
    [-119.6796639222,34.4145537304],
    [-119.677867613,34.4145537304],
    [-119.677867613,34.4160664548],
    [-119.6796639222,34.4160664548],
    [-119.6796639222,34.4145537304]])
    


In [ ]:
# ---------------------------------------
# collect processing information for each scene
times_pre = []
times_class = []
times_post = []
processed = []
reason = []
veg_pixels = [] # number of pixels with ndwi<0.3 and ndwi>0.05
n_pixels = []   # number of non-zero pixels in masked scene

# parameters for GLCM 
distances = [1]     
angles = [0] # East

# counter for scenes queued for processing
t_total = time.time()

# ---------------------------------------
# ---------------------------------------

for itemid in ['ca_m_3411935_se_11_060_20200521']:
    t_alpha = time.time()
    
    # ***********************************************************************************************
    # *************************************** PRE-PROCESSING ****************************************
    # open NAIP scene and clip to coast
    t0 = time.time()
    raster = rf.rioxr_from_itemid(itemid=itemid, reduce_box=reduce_box, reduce_box_crs="EPSG:4326")

    # ---------------------------------------
    # select pixels with data (blacked out portions have 0 on all bands)
    df = rf.raster_as_df(raster.to_numpy(), ['r','g','b','nir'])
    df = df.loc[ (df['nir'] != 0) | (df['r'] != 0) | (df['g'] != 0) | (df['b'] != 0)]
    n_pixels.append(df.shape[0])

    # ---------------------------------------
    # stop if there's no data at intersection
    if df.shape[0] == 0:
        rf.finish_processing('no_data', processed, reason, times_pre, times_class, times_post, veg_pixels, itemid)
        if verbose:
            rf.finish_processing_message('no_data', itemid)

    else:
        # find vegetation pixels to go into model
        # keep ndices of water and low-ndvi pixels
        # add ndvi and ndwi features for each pixel
        t0 = time.time()
        is_veg, water_index, not_veg_index = rf.add_spectral_features(df, 
                                                                      ndwi_thresh = 0.3, 
                                                                      ndvi_thresh = 0.05) 
      # ---------------------------------------
        # stop if there are no vegetation pixels at intersection
        if is_veg.shape[0] == 0:
            rf.finish_processing('no_veg', processed, reason, times_pre, times_class, times_post, veg_pixels, itemid)            
            if verbose:
                rf.finish_processing_message('no_veg', itemid)

        else:
            processed.append('Y')
            reason.append('processed')  
            
            if verbose:
                print('extracted vegetation')            
            
            # ---------------------------------------
            # discard ndwi and ndvi and add date features
            is_veg.drop(['ndwi','ndvi'], axis=1, inplace=True)
            is_veg = rf.add_date_features(is_veg, rf.rioxr_from_itemid(itemid).datetime)


    # *************************************************************************************************
    # ******************************** CREATE R,G,B,NIR AUXILIARY RASTERS *****************************
            t0 = time.time()
            # make auxiliary spectral rasters from clipped NAIP 
            band_names = ['r_', 'g_', 'b_', 'nir_']
            tags = ['_entrs','_avgs']
            window_fps = []
            window_cols = []

            for name, band in zip(band_names,range(1,5)):
                rast_name = name+itemid
                sr.entropy_raster(raster=raster, band=band, rast_name=rast_name, n=entropy_r)
                sr.avg_raster(raster=raster, band=band, rast_name=rast_name,n=box_side)

                for tag in tags:
                    window_fps.append(os.path.join(os.getcwd(), 'temp', rast_name + tag + '.tif'))        
                    #window_cols.append( name.replace('_','')+tag.replace('s',''))
                    window_cols.append( name.replace('_','')+tag.replace('s',str(entropy_r)))                    
            if verbose:
                print('created R,G,B,NIR auxiliary rasters (avgs,entr)', time.time()-t0)
            

            # *************************************************************************************
            # ******************************** CREATE TEXTURE RASTERS *****************************            
            y_len = raster.shape[1]
            x_len = raster.shape[2]
            for band_n, band_name in zip(range(4),band_names):
                if band_n != 1:  # no texture features for Green band    
                    contrast = np.zeros((y_len,x_len))
                    correlation = np.zeros((y_len,x_len))

                    for y in range(y_len):
                        for x in range(x_len):
                            window = raster[:, y-entropy_r:y+entropy_r+1, x-entropy_r:x+entropy_r+1]
                            if (window.shape[1] != 2*entropy_r+1) | (window.shape[2] != 2*entropy_r+1):
                                contrast[y,x] = np.nan
                                correlation[y,x] = np.nan                                
                            else:
                                glcm = graycomatrix(window[band_n], distances=distances, angles=angles) # assuming 1 distance and 1 angle
                                contrast[y,x] = graycoprops(glcm, 'contrast')[0,0]
                                correlation[y,x] = graycoprops(glcm, 'correlation')[0,0]

                    sr.save_raster_checkpoints(contrast, crs=raster.rio.crs, transf=raster.rio.transform(), rast_name=band_name+'contrast')
                    sr.save_raster_checkpoints(correlation, crs=raster.rio.crs, transf=raster.rio.transform(), rast_name=band_name+'correlation')

                    window_fps.append(os.path.join(os.getcwd(), 'temp',band_name + 'contrast.tif')) 
                    window_fps.append(os.path.join(os.getcwd(), 'temp',band_name + 'correlation.tif')) 

                    window_cols = window_cols + [band_name+'cont', band_name+'corr']
                    print('finished textures for band ',band_n)

            del contrast, correlation
            gc.collect()

    # *******************************************************************************************
    # *********************** EXTRACT FEATURES FROM AUXILIARY RASTERS ***************************
            window_values = []    
            for fp_aux in window_fps:
                match = rioxr.open_rasterio(fp_aux).squeeze()
                match_vector = match.to_numpy().reshape(match.shape[0]*match.shape[1])
                window_values.append(match_vector)
                os.remove(fp_aux)

            df_window = pd.DataFrame(dict(zip( window_cols, window_values)))

            scene_features = pd.concat([is_veg, df_window.iloc[is_veg.index]], axis=1)

    # **********************************************************************************************
    # *************** REMOVE NA VALUES (PIXELS AT EDGE OF CLIPPED PART OF RASTER) ******************
           # remove these indices from scene_features
            # no need to add them anywhere else, they will be part of the raster's background
            remove = scene_features[scene_features.r_cont.isna() == True].index
            scene_features = scene_features.drop(remove)

            #free memory            
            del df_window, window_values, match_vector, match, remove
            gc.collect()
            
            times_pre.append(time.time()-t0)
    

            # ******************************************************************************
            # ******************************** ORDER FEATURES ****************************** 

            feature_order = [
                             'r', 'r_avg5', 'r_entr5', 'r_cont', 'r_corr', 
                             'g', 'g_avg5', 'g_entr5', 
                             'b', 'b_avg5', 'b_entr5', 'b_cont', 'b_corr', 
                             'nir', 'nir_avg5', 'nir_entr5', 'nir_cont', 'nir_corr', 
                             'month', 'day_in_year']

            scene_features = scene_features[feature_order]
            if verbose:
                print('finished assembling features')

            # convert into dask.array and predict using model
            # da_pixels = da.from_array(np.array(scene_features), chunks=728802)
            # scene_preds = rfc.predict(da_pixels)

            # ---------------------------------------
            # t0 = time.time()
            # preds = scene_preds.compute()
            # times_class.append(time.time() - t0)
            # if verbose:
            #     print('finished classification')

            preds = rfc.predict(np.array(scene_features))

            # ************************************************************************************************
            # *************************************** POST-PROCESSING ****************************************
            # recover pixel indices for iceplant classifications
            t0 = time.time()
            preds_df = pd.DataFrame(preds, 
                                 columns=['is_iceplant'], 
                                 index = scene_features.index)
            is_iceplant_index = preds_df[preds_df.is_iceplant == 1].index.to_numpy()
            non_iceplant_index = preds_df[preds_df.is_iceplant == 0].index.to_numpy()

            # ---------------------------------------
            # reconstruct indices into image
            indices = [non_iceplant_index,
                       is_iceplant_index, 
                       not_veg_index,
                       water_index]
            values = [0,    # values assigned to pixels from each index
                      1,
                      2,
                      3]
            reconstruct = rf.indices_to_image(raster.shape[1], raster.shape[2], indices, values, back_value=100)

            # ---------------------------------------
            # apply median 3x3 filter if needed
            if filter:
                reconstruct = median_filter(reconstruct, size=3)

            times_post.append(time.time() - t0)
            if verbose:
                print('finished post-processing')

            # ************************************************************************************************
            # *************************************** SAVE RASTERS *******************************************  
            if save_rasters:
                if filter_rasters:
                    filename = prefix+'_filter_clip_preds_' + itemid + '.tif'
                else:
                    filename = prefix+'_clip_preds_' + itemid + '.tif'

                with rasterio.open(
                    os.path.join(fp, filename),  # file path
                    'w',           # w = write
                    driver = 'GTiff', # format
                    height = reconstruct.shape[0], 
                    width = reconstruct.shape[1],
                    count = 1,  # number of raster bands in the dataset
                    dtype = rasterio.uint8,
                    crs = raster.rio.crs,
                    transform = raster.rio.transform(),
                ) as dst:
                    dst.write(reconstruct.astype(rasterio.uint8), 1)

            if verbose:
                print('FINISHED: ', itemid)        


    # ************************************************************************************************
    # ************************************ FINAL INFO MESSAGE ***************************************            

    
print('TOTAL TIME: ', (time.time() - t_total)/60, ' mins')

extracted vegetation
created R,G,B,NIR auxiliary rasters (avgs,entr) 0.5228104591369629
finished textures for band  0
finished textures for band  2


In [ ]:
#save times processed and itemids as dataframe
D = { 'itemid': scene_ids,
     'processed': processed,
     'reason':reason,
     'pre_times': times_pre,
     'class_times' : times_class,
     'post_times' : times_post, 
     'processed_pix' : n_pixels }
processing_df = pd.DataFrame(D)

In [ ]:
if save_processing_times:
    fp = os.path.join(os.getcwd(),'processing_results')
    if os.path.exists(fp) == False:
        os.mkdir(fp)


    if filter_rasters:
        filename = prefix+'_filter_clip_processing_results_' + str(year) + '.csv'
    else:
        filename = prefix+'_clip_processing_results_' + str(year) + '.csv'

    processing_df.to_csv(os.path.join(fp, filename ), index=False)